## Currently, this file will not be used until I can find a better way to use it.

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained(model_path)
    #model = AutoModelForMaskedLM.from_pretrained(model_path, attn_implementation="eager")
    #assamese_tokens = tokenizer(masked_texts, padding="longest", return_tensors="pt")
    #outputs = model(assamese_tokens["input_ids"],assamese_tokens["attention_mask" ],  output_attentions=True) 
    

In [2]:
import http.client

conn = http.client.HTTPSConnection("api.two.ai")

conn.request("GET", "/health")

res = conn.getresponse()
data = res.read()

In [3]:
kfile = open("access_items/sutra_api_key.txt", "r")
sutra_key = kfile.readline()
kfile.close()

In [7]:
import http.client

conn = http.client.HTTPSConnection("api.two.ai")

payload = ("{\n  \"model\": \"sutra-v2\",\n"
            +"  \"messages\": [\n    {\n      \"role\": \"system\",\n"
            + "      \"content\": \"Perform a cloze task.\"\n    },\n  " 
            + "  {\n      \"role\": \"user\",\n"
            + "      \"content\": \"পৃথিৱীখন ইমান <mask>.\"\n    }\n  ],\n  "
            + "\"max_tokens\": 100,\n  \"echo\": true,\n  \"logprobs\": 1,\n  \"temperature\": 0.3,\n  \"stream\": false,\n  \"extra_body\": {\n    \"online_search\": false\n  }\n}")

print(payload)

payload = payload.encode('utf-8')

headers = { 'Content-Type': "application/json", "Authorization": sutra_key }

conn.request("POST", "/v2/chat/completions", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{
  "model": "sutra-v2",
  "messages": [
    {
      "role": "system",
      "content": "Perform a cloze task."
    },
    {
      "role": "user",
      "content": "পৃথিৱীখন ইমান <mask>."
    }
  ],
  "max_tokens": 100,
  "echo": true,
  "logprobs": 1,
  "temperature": 0.3,
  "stream": false,
  "extra_body": {
    "online_search": false
  }
}
{"id":"prjaKym93r1LK3ys6yTpGn","object":"chat.completion","created":1745798018,"model":"sutra-v2","choices":[{"index":0,"message":{"role":"assistant","content":"পৃথিৱীখন ইমান সুন্দৰ।","refusal":null,"annotations":[]},"logprobs":null,"finish_reason":"stop"}]}


In [33]:
from transformers import AutoTokenizer, XLMRobertaForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = XLMRobertaForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-base")

inputs = tokenizer("The capital of France is <mask> .", return_tensors="pt")

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of <mask>
mask_token_indexes = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)

print(mask_token_indexes)

(tensor([6]),)


In [35]:

for mask_token_index in mask_token_indexes[0]:
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    print(tokenizer.decode(predicted_token_id))

Paris
